In [3]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Dict, List, Union

from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

load_dotenv()
chat_key = os.getenv("AZURE_OAI_KEY", "")
chat_model = os.getenv("AZURE_OAI_MODEL_NAME", "")
chat_endpoint = os.getenv("AZURE_OAI_ENDPOINT", "")
chat_api_version = os.getenv("AZURE_OAI_API_VER", "")
chat_deploy = os.getenv("AZURE_OAI_DEPLOY_NAME", "")

keyword_model = os.getenv("AZURE_OAI_KEYWORD_MODEL_NAME", "")

search_key = os.getenv("AZURE_SEARCH_KEY", "")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT", "")
search_index = os.getenv("AZURE_SEARCH_INDEX_NAME", "")

chat_client = AzureOpenAI(
    api_version=chat_api_version,
    azure_endpoint=chat_endpoint,
    api_key=chat_key,
)

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=search_index,
    credential=AzureKeyCredential(search_key)
)

def extract_keyword_from_query(query_text:str, OAI_client:AzureOpenAI, keyword_model : str) -> List[str]:
    keyword_prompt = f"""Query: {query_text}
    
    위 Query에서 주요 키워드를 추출하세요.
    인물명, 지명, 제도명, 사건명 등 사용자의 질문에 적절한 답변을 할 수 있는 문서를 찾아 질문에 사용할 수 있도록 핵심적인 키워드를 중심으로 최대 5개까지 추출하세요.

    

    추출 조건:
    1. 핵심적인 키워드만 추출
    2. 고유명사를 우선적으로 선택
    3. 검색 가능한 구체적인 용어 선택
    4. 중복을 피하고 중요도 순으로 정렬
    5. 제공된 Query에 존재하지 않는 내용은 금지됨

    키워드 목록 (JSON 배열 형태로만 응답):"""
    try:
        keyword_response = OAI_client.chat.completions.create(
            model=keyword_model,
            messages=[
                {"role": "system", "content": "당신은 한국사 키워드 추출 전문가입니다. 정확한 JSON 배열 형태로만 응답하세요."},
                {"role": "user", "content": keyword_prompt}
            ],
            temperature=0.3,
            max_tokens=200
        )
        
        keywords_text = keyword_response.choices[0].message.content.strip()
        if keywords_text[0] == "[" and keywords_text[-1] == "]":
            keywords = json.loads(keywords_text)
            return keywords if isinstance(keywords, list) else []
        else:
            print(f"키워드 추출 오류 {keywords_text}")
            raise Exception
    
    except Exception as e:
        print(f"키워드 추출 오류: {e}")
        return []
    
def extract_keywords_from_response(response_text: str, OAI_client:AzureOpenAI, keyword_model:str) -> List[str]:
    """응답에서 키워드 추출"""
    keyword_prompt = f"""다음 텍스트에서 주요 키워드를 추출하세요.
    인물명, 지명, 제도명, 사건명 등 사용자가 답변의 내용이 적절히 생성되었는지 판단할 수 있는 키워드를 중심으로 최대 5개까지 추출하세요.

    텍스트: {response_text}

    추출 조건:
    1. 핵심적인 키워드만 추출
    2. 고유명사를 우선적으로 선택
    3. 검색 가능한 구체적인 용어 선택
    4. 중복을 피하고 중요도 순으로 정렬
    5. 제공된 텍스트에 존재하지 않는 내용은 금지됨

    키워드 목록 (JSON 배열 형태로만 응답):"""

    try:
        keyword_response = OAI_client.chat.completions.create(
            model=keyword_model,
            messages=[
                {"role": "system", "content": "당신은 한국사 키워드 추출 전문가입니다. 정확한 JSON 배열 형태로만 응답하세요."},
                {"role": "user", "content": keyword_prompt}
            ],
            temperature=0.3,
            max_tokens=200
        )
        
        keywords_text = keyword_response.choices[0].message.content.strip()
        if keywords_text[0] == "[" and keywords_text[-1] == "]":
            keywords = json.loads(keywords_text)
            return keywords if isinstance(keywords, list) else []
        else:
            print(f"키워드 추출 오류 {keywords_text}")
            raise Exception
    
    except Exception as e:
        print(f"키워드 추출 오류: {e}")
        return []

def format_sources(documents: List[Dict]) -> List[str]:
    """문서 목록을 출처 형태로 포맷팅"""
    sources = []
    for doc in documents:
        # source(파일명)을 주 출처로 사용
        if doc.get("source"):
            source_info = doc["source"]
            
            # chunk_id가 있으면 추가 정보로 포함
            if doc.get("chunk_id"):
                # chunk_id에서 페이지 정보 추출 시도
                chunk_id = doc["chunk_id"]
                if "pages_" in chunk_id:
                    page_num = chunk_id.split("pages_")[-1]
                    source_info += f" (페이지 {page_num})"
                else:
                    source_info += f" (ID: {chunk_id})"
            
            sources.append(source_info)
        
        # source가 없으면 chunk_id라도 사용
        elif doc.get("chunk_id"):
            sources.append(f"문서 ID: {doc['chunk_id']}")
    
    return list(set(sources))  # 중복 제거

def get_suggestion(return_type: str, search_client:SearchClient) -> List[str]:
    """DB 기반 제안 기능"""
    result = []
    if return_type == "query":
        try:
            # DB에서 인기 검색어나 추천 쿼리를 가져오는 로직
            # 현재는 Azure Search에서 자주 검색되는 용어들을 기반으로 생성
            popular_searches = search_client.search(
                search_text="*",
                top=50,
                select=["title", "content"],
                include_total_count=True
            )
            
            # 검색 결과에서 자주 등장하는 주제들을 추출하여 쿼리 형태로 변환
            query_suggestions = []
            common_topics = ["세종대왕", "조선시대", "임진왜란", "영조", "정조"]
            
            for topic in common_topics:
                topic_results = search_client.search(
                    search_text=topic,
                    top=1
                )
                for result in topic_results:
                    if result.get("title"):
                        query_suggestions.append(f"{topic}에 대해 더 자세히 알려주세요")
                        break
            
            result = query_suggestions[:5] if query_suggestions else [
                "조선시대에 대해 궁금한 것이 있으면 언제든 물어보세요!"
            ]
            
        except Exception as e:
            print(f"쿼리 제안 생성 오류: {e}")
            result = ["조선시대 역사에 대해 궁금한 점을 물어보세요."]
        
        return result
        
    elif return_type == "keyword":
        try:
            # DB에서 인기 키워드를 추출
            search_results = search_client.search(
                search_text="*",
                top=100,
                select=["content", "title"]
            )
            
            # 키워드 빈도 분석을 위한 간단한 로직
            keyword_counts = {}
            common_keywords = ["세종대왕", "한글창제", "조선시대", "과거제도", 
                             "임진왜란", "이순신", "영조", "탕평책", "정조", "규장각"]
            
            for keyword in common_keywords:
                try:
                    count_results = search_client.search(
                        search_text=keyword,
                        include_total_count=True
                    )
                    keyword_counts[keyword] = count_results.get("@odata.count", 0)
                except:
                    keyword_counts[keyword] = 0
            
            # 빈도순으로 정렬하여 상위 키워드 반환
            sorted_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)
            result = [keyword for keyword, count in sorted_keywords[:10]]
            
        except Exception as e:
            print(f"키워드 제안 생성 오류: {e}")
            result = ["세종대왕", "조선시대", "임진왜란"]
        
        return result
    else:
        raise ValueError("Allowed return type is ['query', 'keyword'].")

def get_text_completion_result(
        Query: Dict[str, str], 
        OAI_Client: AzureOpenAI,
        is_verify: bool = False 
    ) -> Dict[str, Union[List, str]]:
    """
    챗봇 응답 생성 메인 함수
    
    Args:
        Query: {"query": "사용자 질문", "context": "이전 대화 컨텍스트(선택)"}
        is_verify: True면 고증 모드, False면 창작 모드
        OAI_Client: Azure OpenAI 클라이언트
    
    Returns:
        {
            "query_suggestions": [...],  # 컨텍스트 없을 때만
            "response": "AI 응답",
            "keywords": ["키워드1", ...],
            "sources": ["출처1", ...]
        }
    """
    
    user_query = Query.get("query", "")
    context = Query.get("context", "")
    
    result = {
        "response": "",
        "keywords": [],
        "sources": []
    }
    
    # 컨텍스트가 없으면 쿼리 제안 추가
    if not context and not user_query:
        result["query_suggestions"] = get_suggestion("query", search_client)
        result["response"] = "안녕하세요! 저는 역사적 사료 기반의 역사 AI입니다. 역사에 대한 궁금한 점을 물어보세요."
        return result
    
    if not user_query:
        result["response"] = "질문을 입력해주세요."
        return result
    
    try:
        # 1. 관련 문서 검색
        doc_search_keywords = extract_keyword_from_query(user_query, chat_client, keyword_model)
        relevant_docs = get_relevant_documents(" ".join(doc_search_keywords), search_client)
        
        # 2. 컨텍스트 구성
        context_text = ""
        if relevant_docs:
            context_text = "\n".join([
                f"[출처: {doc['title']}]\n{doc['content']}" 
                for doc in relevant_docs[:3]
            ])
        
        # 3. 시스템 프롬프트 설정 - SHOULD IMPLEMENT
        system_prompt = "당신은 역사 전문가입니다. 사용자의 입력과 고증 혹은 창작 여부에 따라 입력에 대한 고증을 진행하거나 사실 기반 사건/인물들을 이용하여 창의적인 내용을 생성해야 합니다."
        
        # 4. 사용자 프롬프트 구성
        user_prompt = f"""
            관련 자료:
            {context_text}

            사용자 질문: {user_query}

            위 자료를 바탕으로 질문에 답변해주세요.
        """
        
        # 5. 모드에 따른 파라미터 설정
        temperature = 0.3 if is_verify else 0.7
        max_tokens = 1000 if is_verify else 1200
        
        # 6. OpenAI API 호출
        response = OAI_Client.chat.completions.create(
            model=chat_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        ai_response = response.choices[0].message.content
        
        # 7. 키워드 추출
        # keywords = extract_keywords_from_response(ai_response, chat_client, keyword_model)
        
        # 8. 출처 정보 구성
        sources = format_sources(relevant_docs)
        
        result.update({
            "response": ai_response,
            "keywords": doc_search_keywords, # origin -> resp keyword, temp change to doc_search_kwd
            "sources": sources
        })
        
    except Exception as e:
        print(f"응답 생성 오류: {e}")
        result["response"] = "죄송합니다. 일시적인 오류가 발생했습니다. 다시 시도해주세요."
    
    return result

def get_relevant_documents(query: str, search_client:SearchClient, top_k: int = 5) -> List[Dict]:
    """Azure Search를 통한 관련 문서 검색"""
    try:
        search_results = search_client.search(
            search_text=query,
            top=top_k,
            include_total_count=True
        )
        
        documents = []
        for result in search_results:
            documents.append({
                "content": result.get("chunk", ""),  # chunk를 content로 매핑
                "source": result.get("title", ""),  # title(파일명)을 source로 매핑
                "title": [],  # 빈 배열로 설정
                "king": [],  # 빈 배열로 설정
                "date": [],  # 빈 배열로 설정
                "score": result.get("@search.score", 0),
                "reranker_score": result.get("@search.rerankerScore", 0),
                "highlights": result.get("@search.highlights", {}),
                "captions": result.get("@search.captions", [])
            })
        
        return documents
    except Exception as e:
        print(f"문서 검색 오류: {e}")
        return []

In [4]:
resssss = get_text_completion_result({"query":"삼국시대에 해당하는 각 나라의 발전과정에 대해 알려줘"}, chat_client, is_verify=True)
resssss

{'response': '삼국시대의 각 나라별 발전 과정은 다음과 같습니다.\n\n1. 고구려\n- 영역 확장: 태조왕과 미천왕(4세기 초 낙랑군 축출) 등의 활약으로 영토를 넓힘.\n- 체제 정비: 소수림왕 때 불교를 수용하고 태학을 설립하였으며 율령을 반포하여 국가 체제를 정비함.\n- 국력 확대: 광개토 대왕은 백제를 공격하여 한강 이북 지역을 차지하고, 신라에 침입한 왜를 격퇴함. 장수왕 때는 남진 정책을 추진하여 평양으로 천도하고 백제를 공격, 한성을 함락시키고 한강 유역을 차지함. 광개토 대왕릉비와 충주 고구려비에서 고구려의 팽창을 확인할 수 있음.\n\n2. 백제\n- 체제 정비: 고이왕 때 관등제를 마련하고, 침류왕 때 불교를 수용함.\n- 국력 확대: 근초고왕 때 마한의 여러 소국을 복속시키고 고구려 평양성을 공격하였으며, 동진과 왜 등과 대외 교류를 활발히 함.\n- 위기 및 중흥 노력: 5세기 후반 고구려의 공격으로 수도 한성을 함락당해 웅진으로 천도함. 무령왕 때 22담로에 왕족을 파견하여 지방 통치를 강화하고, 성왕 때 사비로 천도하여 통치 체제를 정비함.\n\n3. 신라\n- 체제 정비: 내물왕 때 김씨 왕위 계승을 확립하고, 지증왕 때 국호를 ‘신라’로, 왕호를 ‘왕’으로 확정하며 우산국을 복속함. 법흥왕 때 율령을 반포하고 불교를 공인했으며 금관가야를 병합함.\n- 국력 확대: 진흥왕 때 한강 유역을 차지하고 대가야를 정복했으며, 함경도 지역으로 진출함. 단양 신라 적성비와 4개의 순수비를 건립하여 영토 확장을 기록함.\n- 지방 제도: 통일 이후 넓어진 영토와 인구를 효율적으로 다스리기 위해 9주 5소경 체제를 완비함.\n\n이와 같이 삼국은 각기 체제 정비와 영토 확장, 대외 교류 및 내정 강화를 통해 발전해 나갔으며, 고구려와 백제는 주로 북방과 서쪽, 신라는 남부와 동쪽 지역을 중심으로 성장하였습니다.',
 'keywords': ['삼국시대', '고구려', '백제', '신라'],
 'sources': ['2026 수능특강_ 

In [5]:
qt = "통일 신라의 독서삼품과에 대해 설명하고, 그것이 통일신라에 어떠한 기여를 했는지 알려줘."
query = {"query" : f"{qt}"}
resp = get_text_completion_result(query, chat_client, True)
resp

{'response': '통일 신라의 독서삼품과는 원성왕 때 처음으로 시행된 관리 선발 시험 제도입니다. 이 제도는 『춘추좌씨전』, 『예기』, 『문선』 등의 유교 경전을 읽고 그 뜻을 능통하게 이해하는 정도에 따라 사람을 상품(상품)부터 하품(하품)까지 등급으로 나누어 관리로 등용하는 방식이었습니다. 특히 5경과 3사, 제자백가서에 모두 능통한 자는 등급을 뛰어넘어 선발하기도 하였습니다.\n\n독서삼품과는 국학의 기능을 강화하고 유학 교육을 체계화하는 데 기여하였으며, 유학의 보급에도 중요한 역할을 하였습니다. 이를 통해 통일 신라는 유학을 정치 이념으로 삼고, 관리 선발에 유교 경전의 이해를 기준으로 삼아 국가 통치 체제의 질적 향상을 도모하였습니다. 하지만 진골 귀족들의 반대로 인해 제도의 효과가 제한적이었고, 완전한 성과를 거두지는 못했습니다.\n\n요약하면, 독서삼품과는 통일 신라에서 유학 교육과 관료 선발 제도를 제도화하여 유교적 정치 이념을 강화하고 인재 양성에 기여한 중요한 제도였습니다.',
 'keywords': ['통일 신라', '독서삼품과', '제도', '기여', '통일신라'],
 'sources': ['2026 수능특강_ 한국사.pdf']}

In [ ]:
{'response': '통일 신라의 독서삼품과는 원성왕 때 처음으로 시행된 관리 선발 시험 제도입니다. '
'이 제도는 『춘추좌씨전』, 『예기』, 『문선』 등의 유교 경전을 읽고 그 뜻을 능통하게 이해하는 '
'정도에 따라 사람을 상품(상품)부터 하품(하품)까지 등급으로 나누어 관리로 등용하는 방식이었습니다. '
'특히 5경과 3사, 제자백가서에 모두 능통한 자는 등급을 뛰어넘어 선발하기도 하였습니다.\n\n독서삼품과는'
' 국학의 기능을 강화하고 유학 교육을 체계화하는 데 기여하였으며, 유학의 보급에도 중요한 역할을 하였습니다.'
' 이를 통해 통일 신라는 유학을 정치 이념으로 삼고, 관리 선발에 유교 경전의 이해를 기준으로 삼아 국가 통치'
' 체제의 질적 향상을 도모하였습니다. 하지만 진골 귀족들의 반대로 인해 제도의 효과가 제한적이었고, 완전한'
' 성과를 거두지는 못했습니다.\n\n요약하면, 독서삼품과는 통일 신라에서 유학 교육과 관료 선발 제도를 제도화하여'
' 유교적 정치 이념을 강화하고 인재 양성에 기여한 중요한 제도였습니다.',

In [ ]:
{'response': '삼국시대의 각 나라 발전과정은 다음과 같습니다.\n\n1. 고구려\n'
'- 초기 발전: 태조왕과 미천왕(4세기 초)이 낙랑군을 축출하며 영역을 확장하였고,'
' 광개토 대왕 때 백제 공격으로 한강 이북 지역을 차지하였으며, 신라에 침입한 왜를 격퇴하는 등 국력을 크게 확대하였다.'
'\n- 체제 정비: 소수림왕 때 불교를 수용하고 태학을 설치하였으며 율령을 반포하여 국가 체제를 정비하였다.\n-'
' 국력 확대 지속: 장수왕 때 평양으로 천도하고 남진 정책을 추진하여 한성 함락과 한강 유역 장악을 이루었다.\
    n\n2. 백제\n- 초기 발전: 고이왕이 관등제를 마련하고, 침류왕 때 불교를 수용하였다.\n- '
    '국력 확대: 근초고왕 때 마한의 여러 소국을 복속시키고 고구려 평양성을 공격하는 등 대외 교류와 영토 확장을 활발히 하였다.\n- '
    '위기와 중흥 노력: 5세기 후반 고구려의 공격으로 한성을 함락당해 웅진으로 천도하였으며, 무령왕 때 22담로에 왕족을 파견하여 지방 통치를 강화하였다. '
    '성왕 때 사비로 천도하고 통치 체제를 정비하였다.\n\n3. 신라\n- 초기 발전: 내물왕 때 김씨 왕위 계승을 확립하고, 지증왕이 국호 ‘신라’와 왕호 ‘왕’을 확정하며 우산국을 복속하였다.\n- '
    '체제 정비: 법흥왕이 율령을 반포하고 불교를 공인하였으며 금관가야를 병합하였다.\n- 국력 확대: 진흥왕 때 한강 유역을 차지하고 대가야를 정복하였으며, 함경도 지역으로 진출하였다. '
    '또한 단양 신라 적성비와 4개의 순수비를 건립하였다.\n- 통일 이후: 문무왕과 신문왕 때 왕권을 강화하고 귀족 세력을 제압하였으며, 녹읍을 폐지하는 등 통치 체제를 정비하였다.'
    ' 9주 5소경 지방 행정 체제를 완비하였다.\n\n이와 같이 삼국은 각기 체제를 정비하고 국력을 확장하며 한반도에서 경쟁과 발전을 거듭하였습니다.',
 'keywords': ['삼국시대', '고구려', '백제', '신라'],
 'sources': ['2026 수능특강_ 한국사.pdf']}